In [ ]:
#Import necessary modules and libraries
import os
import numpy as np
import pandas as pd
import joblib
from dataclasses import dataclass
from pathlib import Path
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from src.constants import *
from src.utils.common import read_yaml, create_directories, create_json
from src.logging import logger


In [ ]:
#Get the current working directory
%pwd

In [ ]:
#Change the current working directory to the parent directory
os.chdir("../")
%pwd

In [ ]:
#Configuration class for model evaluation
@dataclass(frozen = True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    metrics_file_path: Path
    all_params: dict
    target_column:str

In [ ]:
#Configuration manager class to handle configurations
class ConfigurationManager:

    #Initialization method to read config, params, and schema files
    def __init__(
        self,
        config_filePath = CONFIG_FILE_PATH,
        params_filePath = PARAMS_FILE_PATH,
        schema_filePath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filePath)
        self.params = read_yaml(params_filePath)
        self.schema = read_yaml(schema_filePath)

        create_directories([self.config.artifacts_root])

    #Method to get model evaluation configuration
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir = config.root_dir,
            test_data_path = config.test_data_path,
            model_path = config.model_path,
            metrics_file_path = config.metrics_file_path,
            all_params = params,
            target_column = schema.name
        )
        return model_evaluation_config


In [ ]:
#Model evaluation class to evaluate the trained model (Model Evaluation component)
class ModelEvaluation   :
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
    
    #Method to evaluate the model
    def evaluate_metrics(self, actual, predicted):
        rmse = np.sqrt(mean_squared_error(actual, predicted))
        mae = mean_absolute_error(actual, predicted)
        r2 = r2_score(actual, predicted)

        return rmse, mae, r2 
    
    #Method to save evaluation results
    def save_results(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        X_test = test_data.drop([self.config.target_column], axis=1)
        y_test = test_data[self.config.target_column]

        # Predicting the values using the trained model
        predicted_qualities = model.predict(X_test)

        # Calculating evaluation metrics
        rmse, mae, r2 = self.evaluate_metrics(y_test, predicted_qualities)

        # Saving the evaluation metrics to a JSON file
        scores = {"RMSE": rmse, "MAE": mae, "R2_Score": r2}
        create_json(path = Path(self.config.metrics_file_path), data = scores)



In [ ]:
#Pipeline to execute the model evaluation process
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config = model_evaluation_config)
    model_evaluation.save_results()
except Exception as e:
    logger.exception(e)
    raise e